### This is designed to merge the single papers together to assemble one bigger data-csv that we can at some point use for model training

# General Notes: 
- To execute the notebook, you frist need to produce the .csv from the underlying papers
- I try to convert and select for ENSG gene rows as we got  datasets from Flo with 60k entries which most probably will be me mayjority


In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from scipy.io import mmread
from utils.helper import manhattanplot, violinplot_overall, scatter_plot, get_negative_values, METADATA_COLS, reorder_columns_by_metadata_and_gene_counts, convert_gene_names_to_ensembl
from filepath import *

In [ ]:
### Wimmers 2023 (NOT INCLUDED)
#NOTE This is 1 of 2 Datasets from this paper. 
#NOTE Also this datasets should be clarified with Flo first. It´s PBMC but not clear from whom ? (maybe GSE239799 ??? but also different)
wimmers_2_df = pd.read_csv(PATH_WIMMER2023)
print(wimmers_2_df.shape)
wimmers_2_df.head()
PLATFORM_GEO_ID_wimmers_23 = "GPL24676"
PLATFORM_DESCRIPTION_wimmers_23 = "Illumina NovaSeq 6000 (Homo sapiens)"
METHOD_wimmers_23 = "RNAseq"
#TODO in the data itself PBMC is mentioned but https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE239787 it is whole blood 
TYPE_wimmers_23 = "blood"
study_accession_wimmers_23 = "???"
desease_wimmers_23 = "SARS-CoV-2 / healthy"

In [ ]:
### Wimmers 2023 
#NOTE This is 2 of 2 Datasets from this paper. 
wimmers_GEO239787_df = pd.read_csv(PATH_WIMMER2023, index_col=0)
print(wimmers_GEO239787_df.shape)
wimmers_GEO239787_df = wimmers_GEO239787_df.rename(columns={
  'ID':'subject_accession', 
  'geo_accession':'data_accession',
  'characteristics_ch1.4':'age',
  'characteristics_ch1.3':'gender',
  'Status':'desease', 
  'contact_city':'origin',
  'library_strategy':'METHOD', 
  'source_name_ch1':'TYPE',
  'platform_id':'PLATFORM_GEO_ID',
  'instrument_model':'PLATFORM_DESCRIPTION',
})
wimmers_GEO239787_df['study_accession']='GSE239787'
wimmers_GEO239787_df['origin']='Contact City: Palo Alto'
wimmers_GEO239787_df['age']=wimmers_GEO239787_df['age'].str.replace('age, months: ', '').astype(float)
wimmers_GEO239787_df['age']=wimmers_GEO239787_df['age']/12

wimmers_GEO239787_df_reduced = reorder_columns_by_metadata_and_gene_counts(wimmers_GEO239787_df)

print(wimmers_GEO239787_df_reduced.shape)
wimmers_GEO239787_df_reduced.tail()

In [ ]:
# 2020 Arunachalam, Wimmers
#NOTE migh had some issues in the prep df file, but the GEO actually did a good job 
#NOTE I can probably add 4 more subjects but want to clarify with flo first
aranachalam_2020_GSE152418_df = pd.read_csv(PATH_ARUNACHALAM2020_FINALDESTINATION, index_col=0)
print(aranachalam_2020_GSE152418_df.shape)
aranachalam_2020_GSE152418_df = aranachalam_2020_GSE152418_df.rename(columns={
  'title':'subject_accession', 
  'geo_accession':'data_accession',
  'Age':'age',
  'characteristics_ch1.2':'gender',
  'characteristics_ch1.3':'desease', 
  'contact_city':'origin',
  'library_strategy':'METHOD', 
  'source_name_ch1':'TYPE',
  'instrument_model':'PLATFORM_DESCRIPTION',
  'platform_id':'PLATFORM_GEO_ID',
})
aranachalam_2020_GSE152418_df['study_accession'] = 'GSE152418'
aranachalam_2020_GSE152418_df = reorder_columns_by_metadata_and_gene_counts(aranachalam_2020_GSE152418_df)
print(aranachalam_2020_GSE152418_df.shape)
aranachalam_2020_GSE152418_df.head()

In [ ]:
# 2021 Thair COVID 
# NOTE 

thair_2021_GSE152641_df = pd.read_csv(PATH_THAIR2021, low_memory=False, index_col=0)
print(thair_2021_GSE152641_df.shape)
thair_2021_GSE152641_df = thair_2021_GSE152641_df.rename(columns={
  'title':'subject_accession', 
    'geo_accession':'data_accession',
    'characteristics_ch1.3':'age',
    'characteristics_ch1.4':'gender',
    'characteristics_ch1':'desease',
  'contact_city':'origin',
  'library_strategy':'METHOD',
  'characteristics_ch1.2':'TYPE',
  'instrument_model':'PLATFORM_DESCRIPTION',
  'platform_id':'PLATFORM_GEO_ID',
})
thair_2021_GSE152641_df['study_accession'] ='GSE152641'
thair_2021_GSE152641_ordered_df = reorder_columns_by_metadata_and_gene_counts(thair_2021_GSE152641_df)
print(thair_2021_GSE152641_ordered_df.shape)
thair_2021_GSE152641_ordered_df.head()

In [ ]:
# 2021 Aranachalam VACCINE
#NOTE Had to convert Gene names to ENSEMBL which sometimes results in a not found version -> Flo how to handle 
#NOTE Vaccine study, i just took the Baseline (blood before the vaccine was given)

import copy

aranachalam_2021_GSE169159_df = pd.read_csv(PATH_ARANACHALAM2021)
print(aranachalam_2021_GSE169159_df.shape)
aranachalam_2021_GSE169159_df = aranachalam_2021_GSE169159_df.rename(columns={
  'title':'subject_accession', 
  'geo_accession':'data_accession',
  'age':'age',
    'sex':'gender',
  'contact_city':'origin',
  'library_strategy':'METHOD',
  'source_name_ch1':'TYPE',
  'instrument_model':'PLATFORM_DESCRIPTION',
  'platform_id':'PLATFORM_GEO_ID',
})
aranachalam_2021_GSE169159_df['desease']='Healthy'
aranachalam_2021_GSE169159_df['study_accession']='GSE169159'

## This is weird as the overwrite of a subset of columns of a dataframe seems not to work. I tried stuff in different patterns here
print(f"First gene: {aranachalam_2021_GSE169159_df.columns.get_loc('FGR')}")
gene_names_to_convert = aranachalam_2021_GSE169159_df.iloc[:, 38:-2].copy().columns
aranachalam_2021_GSE169159_ordered_df = reorder_columns_by_metadata_and_gene_counts(aranachalam_2021_GSE169159_df, 
                                                                                    metadata_cols=METADATA_COLS,
                                                                                    gene_columns=gene_names_to_convert)
print(f"First gene: {aranachalam_2021_GSE169159_ordered_df.columns.get_loc('FGR')}")
converted_gene_names = convert_gene_names_to_ensembl(aranachalam_2021_GSE169159_ordered_df.iloc[:, 11:].copy().columns)

# Now i want to overwrite every column of the dataset (also i dont need to use loc or iloc)
all_dataset_new_columns = METADATA_COLS + converted_gene_names
aranachalam_2021_GSE169159_ordered_df.columns = all_dataset_new_columns

print(aranachalam_2021_GSE169159_ordered_df.columns[60])
print(converted_gene_names[49]) # as i need to add the meta cols ... 

print(aranachalam_2021_GSE169159_ordered_df.shape)
aranachalam_2021_GSE169159_ordered_df.tail()

### Concat together

This will contain unprocessed concatinated file with raw counts from rna seq. 
### TODO´s:
- preprocessing
- unify entries for everything
- NA and NAN in gene counts 
- douplicates (as there will also be longitudinal data) (check for "draw as well")


In [ ]:
concatinated_df = pd.concat([aranachalam_2020_GSE152418_df, 
                             wimmers_GEO239787_df_reduced, 
                             thair_2021_GSE152641_df, 
                             aranachalam_2021_GSE169159_ordered_df])
print(concatinated_df.shape)
concatinated_df.head()


